In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.models import load_model, Model
from keras.callbacks import EarlyStopping
import keras.backend as K
import numpy as np

# our stuff
from synth import *
from utils import *
from const import Tx, Ty, n_a, n_s, input_vocab, output_vocab, output_vocab_inv

import matplotlib.pyplot as plt
%matplotlib inline

/home/opyate/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# check GPU
from tensorflow.python.client import device_lib
[print(x.name) for x in device_lib.list_local_devices()]

/device:CPU:0
/device:GPU:0


[None, None]

In [3]:
# Returns input,output,dt
# Run once to see an example.
load_time()

("twenty five minutes and fifty five seconds past three o'clock in the morning",
 '03:25:55',
 datetime.time(3, 25, 55))

In [4]:
# One-hot representations of load_time()'s input and output
# will be fed into the model.

i, o, _ = load_time()

print('Input:', i)
print('Target:', o)

ioh, ooh = one_hot([i], [o], Tx, Ty, input_vocab, output_vocab)

print('Input shape:', ioh.shape)
print('Target shape:', ooh.shape)
print(*ooh)

Input: 18 hours, 8 mins and 34 seconds
Target: 18:08:34
Input shape: (1, 128, 102)
Target shape: (1, 8, 11)
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


- ioh, "input one-hot", has shape (128,102), i.e. it can accept an input string of up to (or padded to) 128 characters of a possible vocabulary of 102 characters.
- ooh, "output one-hot", has shape (8,11), i.e. the model will predict 8 characters of the form 'hh:mm:ss' of a possible vocabulary of all digits and colon ':'.

In [5]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
# softmax(axis = 1) from utils.py
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes = 1)

In [6]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e.
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas"
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas, a])
    
    return context

In [7]:
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(output_vocab), activation=softmax)

In [8]:
def get_model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    # Step 1: Define pre-attention Bi-LSTM. Remember to use return_sequences=True.
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs.
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    return model

In [9]:
model = get_model(Tx, Ty, n_a, n_s, len(input_vocab), len(output_vocab))

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 102)     0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 128, 128)     85504       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 128, 128)     0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

In [ ]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=2, verbose=0, mode='auto')
data_maker = dataset_generator(Tx, Ty, input_vocab, output_vocab, n_s, batch_size=1000)
model.fit_generator(data_maker, steps_per_epoch=1000, epochs=10, callbacks=[early_stop])

Epoch 1/10
1000/1000 [==============================] - 248s 248ms/step - loss: 7.8540 - dense_2_loss_1: 0.9547 - dense_2_loss_2: 2.2128 - dense_2_loss_3: 0.0590 - dense_2_loss_4: 0.9586 - dense_2_loss_5: 1.4354 - dense_2_loss_6: 0.0838 - dense_2_loss_7: 1.0021 - dense_2_loss_8: 1.1477 - dense_2_acc_1: 0.5225 - dense_2_acc_2: 0.1640 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 0.5982 - dense_2_acc_5: 0.4559 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 0.5852 - dense_2_acc_8: 0.5799
Epoch 2/10
1000/1000 [==============================] - 245s 245ms/step - loss: 2.9524 - dense_2_loss_1: 0.4110 - dense_2_loss_2: 1.6665 - dense_2_loss_3: 6.6373e-04 - dense_2_loss_4: 0.1818 - dense_2_loss_5: 0.3363 - dense_2_loss_6: 0.0015 - dense_2_loss_7: 0.2104 - dense_2_loss_8: 0.1441 - dense_2_acc_1: 0.8201 - dense_2_acc_2: 0.3593 - dense_2_acc_3: 1.0000 - dense_2_acc_4: 0.9487 - dense_2_acc_5: 0.9133 - dense_2_acc_6: 1.0000 - dense_2_acc_7: 0.9378 - dense_2_acc_8: 0.9655
Epoch 3/10
 109/1000 [==>.............

In [ ]:
EXAMPLES = [
    'the time now is noon',
    "it is five o'clock",
    "half past eleven o'clock in the morning",
    "twenty nine minutes and fifty nine seconds past noon",
    "twenty nine minutes and fifty nine seconds past eleven o'clock at night",
    "it is twenty eight minutes to noon"
]

np.set_printoptions(threshold=10)
#print(human_vocab)
#print('human', len(human_vocab))
for example in EXAMPLES:
    print()
    print(example)
    source = string_to_int(example, Tx, human_vocab)
    
    #print(source)
    
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))) #.swapaxes(0,1)
             
    #print(source.shape)
    #print(source)
    
    prediction = model.predict([np.expand_dims(source, axis=0), s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))
    

In [ ]:
np.set_printoptions(threshold=np.nan)
def plot_attention_map2(model, input_vocabulary, inv_output_vocabulary, text, n_s = 128, num = 6, Tx = 30, Ty = 10):
    """
    Plot the attention map.
  
    """
    attention_map = np.zeros((10, 30))
    Ty, Tx = attention_map.shape
    
    s0 = np.zeros((1, n_s))
    #s0 = np.random.rand(1, n_s)
    c0 = np.zeros((1, n_s))
    #c0 = np.random.rand(1, n_s)
    layer = model.layers[num]

    encoded = np.array(string_to_int(text, Tx, input_vocabulary)).reshape((1, 30))
    print('encoded 1')
    print(encoded)
    print('input_vocabulary')
    print(input_vocabulary)
    print('len(input_vocabulary)')
    print(len(input_vocabulary))
    encoded = np.array(list(map(lambda x: to_categorical(x, num_classes=len(input_vocabulary)), encoded)))
    #print('encoded 2')
    #print(encoded)
    
    source = string_to_int(text, Tx, input_vocabulary)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(input_vocabulary)), source))).swapaxes(0,1)

    #print('source')
    #print(source)
    encoded = source
    #print('model.layers[0]')
    #print(dir(model.layers[0]))
    #print('model.inputs')
    #print(dir(model.inputs))
    
    #f = K.function(model.input, layer.output)
    # AttributeError: Layer attention_weights has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.
    
    print('inputs size')
    print(len(model.inputs))
    print('layer functions')
    print(layer.output_shape)
    print(dir(layer))

    f = K.function(model.inputs + [K.learning_phase()], [layer.get_output_at(t) for t in range(Ty)])
    r = f([encoded, s0, c0, 1])
    
    #intermediate_layer_model = Model(inputs=model.input,outputs=[layer.get_output_at(t) for t in range(Ty)])
    #r = intermediate_layer_model.predict([encoded, s0, c0])

    print('\nr')
    print(np.array(r).shape)
    print(r)
    print('\ns0')
    print(s0)
    print('\nc0')
    print(c0)
    
    for t in range(Ty):
        for t_prime in range(Tx):
            attention_map[t][t_prime] = r[t][0,t_prime,0]

    # Normalize attention map
#     row_max = attention_map.max(axis=1)
#     attention_map = attention_map / row_max[:, None]

    prediction = model.predict([encoded, s0, c0])
    
    predicted_text = []
    for i in range(len(prediction)):
        predicted_text.append(int(np.argmax(prediction[i], axis=1)))
        
    predicted_text = list(predicted_text)
    predicted_text = int_to_string(predicted_text, inv_output_vocabulary)
    text_ = list(text)
    
    # get the lengths of the string
    input_length = len(text)
    output_length = Ty
    
    # Plot the attention_map
    plt.clf()
    f = plt.figure(figsize=(8, 8.5))
    ax = f.add_subplot(1, 1, 1)

    # add image
    i = ax.imshow(attention_map, interpolation='nearest', cmap='Blues')

    # add colorbar
    cbaxes = f.add_axes([0.2, 0, 0.6, 0.03])
    cbar = f.colorbar(i, cax=cbaxes, orientation='horizontal')
    cbar.ax.set_xlabel('Alpha value (Probability output of the "softmax")', labelpad=2)

    # add labels
    ax.set_yticks(range(output_length))
    ax.set_yticklabels(predicted_text[:output_length])

    ax.set_xticks(range(input_length))
    ax.set_xticklabels(text_[:input_length], rotation=45)

    ax.set_xlabel('Input Sequence')
    ax.set_ylabel('Output Sequence')

    # add grid and legend
    ax.grid()

    #f.show()
    
    return attention_map
attention_map = plot_attention_map2(model, human_vocab, inv_machine_vocab, "November 17 1993", num = 6, n_s = 128)